In [1]:
import numpy as np
import pandas as pd

# Sudoku Solver

In [2]:
cell_size = 3
line_size = cell_size**2
line = list(range(0, line_size))
cell = list(range(0, cell_size))

initial_grid = pd.DataFrame(data=[
    [0, 6, 0, 0, 5, 0, 0, 2, 0],
    [0, 0, 0, 3, 0, 0, 0, 9, 0],
    [7, 0, 0, 6, 0, 0, 0, 1, 0],
    [0, 0, 6, 0, 3, 0, 4, 0, 0],
    [0, 0, 4, 0, 7, 0, 1, 0, 0],
    [0, 0, 5, 0, 9, 0, 8, 0, 0],
    [0, 4, 0, 0, 0, 1, 0, 0, 6],
    [0, 3, 0, 0, 0, 8, 0, 0, 0],
    [0, 2, 0, 0, 4, 0, 0, 5, 0]
])
initial_grid

,0,1,2,3,4,5,6,7,8
0,0,6,0,0,5,0,0,2,0
1,0,0,0,3,0,0,0,9,0
2,7,0,0,6,0,0,0,1,0
3,0,0,6,0,3,0,4,0,0
4,0,0,4,0,7,0,1,0,0
5,0,0,5,0,9,0,8,0,0
6,0,4,0,0,0,1,0,0,6
7,0,3,0,0,0,8,0,0,0
8,0,2,0,0,4,0,0,5,0


### A. CP-SAT solution (OR-Tools) with integer variables

**Source from** https://github.com/google/or-tools/blob/stable/examples/python/sudoku_sat.py

It is possible to use **integer variables** since CP-SAT supports the `AddAllDifferent()` method.

In [3]:
from ortools.sat.python import cp_model

# Create the model.
model = cp_model.CpModel()

In [4]:
# Variables
grid = {}
for i in line:
    for j in line:
        grid[i, j] = model.NewIntVar(1, line_size, f"grid[{i},{j}]")

In [5]:
# Constraints
for i in line:
    # AllDifferent on rows.
    model.AddAllDifferent(grid[i, j] for j in line)
    # AllDifferent on columns.
    model.AddAllDifferent(grid[j, i] for j in line)

# AllDifferent on cells.
for i in cell:
    for j in cell:
        model.AddAllDifferent(
            grid[i * cell_size + di, j * cell_size + dj] \
                for di in cell for dj in cell
        )

# Initial values.
for i in line:
    for j in line:
        value = initial_grid.loc[i, j]
        if value:
            model.Add(grid[i, j] == value)

In [6]:
# Solve
solver = cp_model.CpSolver()
%time status = solver.Solve(model)

CPU times: user 5.24 ms, sys: 1.52 ms, total: 6.76 ms
Wall time: 5.2 ms


In [7]:
# Print out the solution.
if status == cp_model.OPTIMAL:
    for i in line:
        print([int(solver.Value(grid[i, j])) for j in line])
else:
    print('No solution.')

[8, 6, 1, 4, 5, 9, 7, 2, 3]
[4, 5, 2, 3, 1, 7, 6, 9, 8]
[7, 9, 3, 6, 8, 2, 5, 1, 4]
[2, 1, 6, 8, 3, 5, 4, 7, 9]
[9, 8, 4, 2, 7, 6, 1, 3, 5]
[3, 7, 5, 1, 9, 4, 8, 6, 2]
[5, 4, 7, 9, 2, 1, 3, 8, 6]
[1, 3, 9, 5, 6, 8, 2, 4, 7]
[6, 2, 8, 7, 4, 3, 9, 5, 1]


### B. CP-SAT solution (OR-Tools) with boolean variables

The CP-SAT solution with boolean variables is slower than the one with integer variables.

In [8]:
from ortools.sat.python import cp_model

# Create the model.
model = cp_model.CpModel()

In [9]:
# Variables
grid = {}
for i in line:
    for j in line:
        for k in line:
            grid[i, j, k] = model.NewBoolVar(f"grid[{i},{j},{k}]")

In [10]:
# Constraints
for i in line:
    for j in line:
        # AllDifferent on rows.
        model.Add(sum(grid[i, k, j] for k in line) == 1)
        # AllDifferent on columns.
        model.Add(sum(grid[k, i, j] for k in line) == 1)
        # Only one value in each grid.
        model.Add(sum(grid[i, j, k] for k in line) == 1)

# AllDifferent on cells.
for i in cell:
    for j in cell:
        for k in line:
            model.Add(
                sum(grid[i * cell_size + di, j * cell_size + dj, k] \
                    for di in cell for dj in cell) == 1
            )

# Initial values.
for i in line:
    for j in line:
        value = initial_grid.loc[i, j]
        if value:
            model.Add(grid[i, j, value - 1] == 1)

In [11]:
# Solve
solver = cp_model.CpSolver()
%time status = solver.Solve(model)

CPU times: user 13.6 ms, sys: 2.08 ms, total: 15.7 ms
Wall time: 12.5 ms


In [12]:
def getValue(i, j):
    return [solver.Value(grid[i, j, k]) for k in line].index(1) + 1

# Print out the solution.
if status == cp_model.OPTIMAL:
    for i in line:
        print([getValue(i, j) for j in line])
else:
    print('No solution.')

[8, 6, 1, 4, 5, 9, 7, 2, 3]
[4, 5, 2, 3, 1, 7, 6, 9, 8]
[7, 9, 3, 6, 8, 2, 5, 1, 4]
[2, 1, 6, 8, 3, 5, 4, 7, 9]
[9, 8, 4, 2, 7, 6, 1, 3, 5]
[3, 7, 5, 1, 9, 4, 8, 6, 2]
[5, 4, 7, 9, 2, 1, 3, 8, 6]
[1, 3, 9, 5, 6, 8, 2, 4, 7]
[6, 2, 8, 7, 4, 3, 9, 5, 1]


### C. MIP solution (OR-Tools + SCIP) with boolean variables

The MIP solution with boolean variables is faster than the CP-SAT solution with boolean variables.

In [13]:
from ortools.linear_solver import pywraplp

# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

In [14]:
# Variables
grid = {}
for i in line:
    for j in line:
        for k in line:
            grid[i, j, k] = solver.BoolVar(f"grid[{i},{j},{k}]")

In [15]:
# Constraints
for i in line:
    for j in line:
        # AllDifferent on rows.
        solver.Add(solver.Sum(grid[i, k, j] for k in line) == 1)
        # AllDifferent on columns.
        solver.Add(solver.Sum(grid[k, i, j] for k in line) == 1)
        # Only one value in each grid.
        solver.Add(solver.Sum(grid[i, j, k] for k in line) == 1)

# AllDifferent on cells.
for i in cell:
    for j in cell:
        for k in line:
            solver.Add(
                solver.Sum(grid[i * cell_size + di, j * cell_size + dj, k] \
                    for di in cell for dj in cell) == 1
            )

# Initial values.
for i in line:
    for j in line:
        value = initial_grid.loc[i, j]
        if value:
            solver.Add(grid[i, j, value - 1] == 1)

In [16]:
# Solve
%time status = solver.Solve()

CPU times: user 6.55 ms, sys: 1.11 ms, total: 7.66 ms
Wall time: 7.62 ms


In [17]:
def getValue(i, j):
    return [grid[i, j, k].solution_value() for k in line].index(1) + 1

# Print out the solution.
if status == pywraplp.Solver.OPTIMAL:
    for i in line:
        print([getValue(i, j) for j in line])
else:
    print('No solution.')

[8, 6, 1, 4, 5, 9, 7, 2, 3]
[4, 5, 2, 3, 1, 7, 6, 9, 8]
[7, 9, 3, 6, 8, 2, 5, 1, 4]
[2, 1, 6, 8, 3, 5, 4, 7, 9]
[9, 8, 4, 2, 7, 6, 1, 3, 5]
[3, 7, 5, 1, 9, 4, 8, 6, 2]
[5, 4, 7, 9, 2, 1, 3, 8, 6]
[1, 3, 9, 5, 6, 8, 2, 4, 7]
[6, 2, 8, 7, 4, 3, 9, 5, 1]


### D. MIP solution (PySCIPOpt) with boolean variables

PySCIPOpt is faster than OR-Tools + SCIP.

In [18]:
from pyscipopt import Model

# Create the model.
model = Model("Sudoku")

In [19]:
# Variables
grid = {}
for i in line:
    for j in line:
        for k in line:
            grid[i, j, k] = model.addVar(vtype='B', name=f"grid[{i},{j},{k}]")

In [20]:
# Constraints
for i in line:
    for j in line:
        # AllDifferent on rows.
        model.addCons(sum(grid[i, k, j] for k in line) == 1)
        # AllDifferent on columns.
        model.addCons(sum(grid[k, i, j] for k in line) == 1)
        # Only one value in each grid.
        model.addCons(sum(grid[i, j, k] for k in line) == 1)

# AllDifferent on cells.
for i in cell:
    for j in cell:
        for k in line:
            model.addCons(
                sum(grid[i * cell_size + di, j * cell_size + dj, k] \
                    for di in cell for dj in cell) == 1
            )

# Initial values.
for i in line:
    for j in line:
        value = initial_grid.loc[i, j]
        if value:
            model.addCons(grid[i, j, value - 1] == 1)

In [21]:
# Solve
%time model.optimize()

CPU times: user 5.85 ms, sys: 881 µs, total: 6.73 ms
Wall time: 6.61 ms


In [22]:
def getValue(i, j):
    return [model.getVal(grid[i, j, k]) for k in line].index(1) + 1

# Print out the solution.
if model.getStatus() == 'optimal':
    for i in line:
        print([getValue(i, j) for j in line])
else:
    print('No solution.')

[8, 6, 1, 4, 5, 9, 7, 2, 3]
[4, 5, 2, 3, 1, 7, 6, 9, 8]
[7, 9, 3, 6, 8, 2, 5, 1, 4]
[2, 1, 6, 8, 3, 5, 4, 7, 9]
[9, 8, 4, 2, 7, 6, 1, 3, 5]
[3, 7, 5, 1, 9, 4, 8, 6, 2]
[5, 4, 7, 9, 2, 1, 3, 8, 6]
[1, 3, 9, 5, 6, 8, 2, 4, 7]
[6, 2, 8, 7, 4, 3, 9, 5, 1]
